# House Prices

This project involved building an interactive web app, using Shiny for Python, to display house prices in England and Wales, adding a Choropleth layer to visualise the differences by region. The aim was to demonstrate the Shiny software and investigate trends in house prices over time and across different regions of England and Wales.

##### About the data

Write description here of how I got the data and how it's created/published.

##### Initial planning

- What granularity to use? e.g. county, town, postcode
    - The dataset contains the postcode of each property, and every postcode has a clearly defined area, which should be available online
    - The whole postcode is likely too granular - using the area code or district code should be a good compromise
- How to aggregate price paid by area?
    - Min, Max, Median, Mean
    - Allow user to choose which statistic to show on the map
    - Could also compare these summary statistics between points in time, which would highlight the areas which have seen the greatest changes in price over time

##### Preparing the data

Loading the CSV file into a MySQL database.

~~~~sql
DROP DATABASE IF EXISTS `houseprices`;
CREATE DATABASE `houseprices`;
USE `houseprices`;

CREATE TABLE `pricepaid` (
`unique_id` VARCHAR(100),
`price_paid` DECIMAL,
`deed_date` DATE,
`postcode` VARCHAR(8),
`property_type` VARCHAR(1),
`new_build` VARCHAR(1),
`estate_type` VARCHAR(1),
`saon` VARCHAR(50),
`paon` VARCHAR(50),
`street` VARCHAR(50),
`locality` VARCHAR(50),
`town` VARCHAR(50),
`district` VARCHAR(50),
`county` VARCHAR(50),
`transaction_category` VARCHAR(1),
`linked_data_uri` VARCHAR(1),
PRIMARY KEY (unique_id)
);

SET GLOBAL local_infile=ON;
SET autocommit=0;
SET unique_checks=1;
SET foreign_key_checks=0;

LOAD DATA LOW_PRIORITY 
LOCAL INFILE 'Path/To/Project/pricepaid.csv'
INTO TABLE pricepaid 
CHARACTER SET armscii8
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n' 
(`unique_id`,`price_paid`,`deed_date`,`postcode`,`property_type`,`new_build`,`estate_type`,`saon`,`paon`,`street`,`locality`,`town`,`district`,`county`,`transaction_category`,`linked_data_uri`);
~~~~

I had originally planned to use the full dataset in my Shiny app, however the full table is ~5GB in size with ~29m rows. I chose to get around this by sampling the dataset. Taking a simple random sample of the data would mean that the number of samples from each area would be proportional to the population of that area, so to ensure that each area had an equal number of samples I would use stratified sampling instead.

I needed to choose a level of granularity to which to stratify the data. A UK postcode is made up of 2 parts, the outward code (first part) and inward code (second part), separated by a space. The outward code consists of the postcode area (either 1 or 2 letters) followed by the postcode district (usually 1 or 2 digits). For example, in the postcode PO16 7GZ, PO16 is the outward code (or outcode), PO is the area and 16 is the district.

OutCode and PostcodeArea were added as generated columns to the pricepaid table, along with a Year column and a YearBin column.

~~~~sql
ALTER TABLE pricepaid ADD COLUMN OutCode VARCHAR(4) GENERATED ALWAYS AS substr(postcode, 1, locate(' ', postcode) - 1) STORED;
ALTER TABLE pricepaid ADD COLUMN PostcodeArea VARCHAR(3) GENERATED ALWAYS AS regexp_replace(OutCode, '[0-9]+', '') STORED;
ALTER TABLE pricepaid ADD COLUMN Year INT GENERATED ALWAYS AS year(cast(deed_date as date)) STORED;
ALTER TABLE pricepaid ADD COLUMN YearBin VARCHAR(4) GENERATED ALWAYS AS case when (Year < 2005) then '1995 - 2004' when (Year < 2015) then '2005 - 2014' else '2015 +' end STORED;
~~~~

Creating an index on Outcode and YearBin to speed up the stratified sample query.

~~~~sql
CREATE INDEX OutcodeYearBinIndex ON pricepaid (Outcode, YearBin);
~~~~

Taking a stratified sample of 100 observations for each distinct OutCode and YearBin.

~~~~sql
SELECT t.* FROM
(SELECT pp.*, ROW_NUMBER() OVER (PARTITION BY OutCode, YearBin ORDER BY RAND()) AS SeqNum
FROM pricepaid pp) t
WHERE t.SeqNum <= 100
INTO LOCAL OUTFILE '/Path/To/Project/pricepaidsample.csv'
FIELDS TERMINATED BY ','
ENCLOSED BY '"'
LINES TERMINATED BY '\n';
~~~~

##### Transforming and aggregating the data

Before the data can be used in the Shiny app, it needs to be aggregated by area. Doing this outside the Shiny app and instead reading the aggregated data directly improves the performance of the app.

Additionally, a GeoJSON file needed to be created so that the Shiny app knows where the boundaries of each postcode area are.

First, the sample CSV is read into a Python script:

In [ ]:
from pathlib import Path
import pandas as pd

### Load data
appDir = Path(__file__).parent
print("Importing data...")
dataset = pd.read_csv(appDir / "pricepaidsample.csv", delimiter='\t', header=0, encoding="utf-8")
# remove missing postcodes
dataset = dataset[~dataset['postcode'].isnull()]

A folder containing a GeoJSON file with the polygon coordinates of each Outcode was downloaded online. This is combined into a single GeoJSON file:

In [ ]:
import json

### Get polygon coordinates (GeoJSON) of each PostcodeArea
geojsonDir = appDir / 'districts'
# combine all PostcodeArea datasets, one for each PostcodeArea
geojsonDict = {}
print("Importing geojson files...")
for file in geojsonDir.glob('*.geojson'):
    with open(file, 'r') as f:
        geojsonDict[str(file).split('\\')[-1][:-8]] = json.load(f)
# add id string to link to summary data
for key in geojsonDict.keys():
    geojsonDict[key]['features'][0]['id'] = key
# changing format of geojsonDict to meet required format for Choropleth function
geojsonList = []
uniqueOutcodes = dataset['Outcode'].unique().tolist()
for outcode in geojsonDict.keys():
    features = geojsonDict[outcode]['features']
    if outcode in uniqueOutcodes:
        geojsonList.append(features[0])
geojsonDict = {}
geojsonDict['type'] = 'FeatureCollection'
geojsonDict['features'] = geojsonList

Summary statistics (mean, median, min, max) of the price paid were calculated for each Outcode and YearBin:

In [ ]:
import itertools

### aggregate data import by Outcode and YearBin
summary = dataset.groupby(["Outcode", "YearBin"])['price_paid'].agg(['min', 'max', 'mean', 'median']).reset_index()
# make sure there is a row for every Outcode and YearBin - set aggregate values to null if missing
uniqueYearBins = dataset['YearBin'].unique().tolist()
crossJoin = list(itertools.product(uniqueOutcodes, uniqueYearBins))
crossJoin = pd.DataFrame(crossJoin, columns=["Outcode", "YearBin"])
summary = pd.merge(crossJoin, summary, how="left", on=["Outcode", "YearBin"])

As the final step in the data preparation, the GeoJSON file and summary dataset were exported to the folder so that they can be read into the Shiny app:

In [ ]:
# export as geojson dictionary as json file
with open(appDir / 'OutcodeCoordinates.json', 'w') as fp:
    json.dump(geojsonDict, fp)

# export summary dataframe as csv
summary.to_csv(appDir / 'summary.csv', index=False)

##### Building the Shiny app

The app.py program of the Shiny app consists of 3 main parts:

- importing the data
- building the HTML interface
- defining a server function

###### Importing the data

In [ ]:
from pathlib import Path
import json
import pandas as pd
from ipyleaflet import Map, Choropleth
from shiny import App, Inputs, Outputs, Session, ui, reactive
from shinywidgets import output_widget, render_widget
from branca.colormap import linear

### Load data
appDir = Path(__file__).parent
with open(appDir / 'OutcodeCoordinates.json', 'r') as f:
    outcodeCoordinates = json.load(f)
summary = pd.read_csv(appDir / 'summary.csv')
yearBins = list(summary['YearBin'].unique())

###### Building the HTML interface

In [ ]:
# Nest Python functions to build an HTML interface
app_ui = ui.page_fillable( 
    # Layout the UI with Layout Functions
    # Add Inputs with ui.input_*() functions 
    # Add Outputs with ui.output_*() functions
    ui.layout_sidebar(
        ui.sidebar(
            ui.input_checkbox_group('yearBin', "Time Period", yearBins),
            ui.input_select('statistic', "House Price Summary Statistic", ['mean', 'median', 'min', 'max'], selected='mean', multiple=False),
            ui.input_switch("switch", "Compare Summary Statistic between Time Periods", False)
        ),
        ui.card(
            output_widget("map", width="auto", height="auto")
        )
    ),
    title="UK House Prices Visualisation"
)

This results in a HTML template that looks like this:

![HTMLTemplate](../images/HTMLTemplate.PNG)

###### Defining a server function

Within the server function, the createChoroData function is reactive and will be called every time one of the inputs to the app changes. The functions uses the user inputs the filter the dataset and return a dictionary containing a key for each Outcode and a value corresponding to the summary statistic.

The createChoroData function also contains logic for if more than one time period is selected - in this case the function will calculate the difference between the summary statistics of the earliest and latest time period selected.

The map function creates an ipyleaflet Map object and adds a Choropleth layer to it.

In [ ]:
# Define server
def server(input: Inputs, output: Outputs, session: Session):
    # function to filter the summary dataset and return a lookup dictionary with a key for each Outcode
    @reactive.calc
    def createChoroData():

        # select all time periods if none are selected
        if input.yearBin() == tuple():
            filter = yearBins
        else:
            filter = list(input.yearBin())

        # logic for comparing summary stastics between time periods
        if input.switch():
            minYearBin = filter[0]
            maxYearBin = filter[-1]
            dfMin = summary[summary['YearBin'] == minYearBin][['Outcode', input.statistic()]]
            dfMax = summary[summary['YearBin'] == maxYearBin][['Outcode', input.statistic()]]
            df = pd.merge(dfMin, dfMax, how="inner", on="Outcode")
            df['diff'] = df[input.statistic() + '_y'] - df[input.statistic() + '_x']
            df = df.set_index('Outcode')
            df['decile'] = pd.qcut(df['diff'], 10, labels=False)
            return df['decile'].to_dict()
        # if not comparing time periods then just show summary statistic
        else:
            df = summary[summary['YearBin'].isin(filter)].set_index('Outcode')
            df['decile'] = pd.qcut(df[input.statistic()], 10, labels=False)
            return df['decile'].to_dict()

    ### For each output, define a function that generates the output
    @render_widget  
    def map():
        # create a Map object and add a Choropleth layer to it
        m = Map(center=(54.00366, -2.547855), zoom=5.5, zoom_snap=0.2)

        layer = Choropleth(
                    geo_data=outcodeCoordinates,
                    choro_data=createChoroData(),
                    key_on='id',
                    colormap=linear.viridis,
                    border_color='black',
                    style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
                )
        
        m.add(layer)

        return m

# Call App() to combine app_ui and server() into an interactive app
app = App(app_ui, server)

##### Transforming the data (again)

The Shiny app is now working, however it is not very insightful because (except for a couple of areas in London) every area appears to be the same colour:

![NoTransformation](../images/MapNoTransformation.PNG)

The density plot below, of all of the house prices in the sample, shows what the problem is. The data is highly right skewed, and because the colour map works by assigning a different colour to each quantile, the majority of data has the same colour.

In [ ]:
# code to aggregate SQL data
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load data
appDir = Path(__file__).parent
dataset = pd.read_csv(appDir / "pricepaidsample.csv", delimiter='\t', header=0, encoding="utf-8")
# remove missing postcodes
dataset = dataset[~dataset['postcode'].isnull()]

ax = sns.kdeplot(data=dataset, x='price_paid')
ax.set_xlabel('Price Paid')
plt.show()

![DensityPlot](../images/DensityPlot.PNG)

To resolve this, I first tried using the QuantileTransformer function from scikit-learn to mathematically transform the data to follow a more normal distribution.

In [ ]:
from sklearn.preprocessing import QuantileTransformer
import numpy as np

quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
quantile_summary = quantile_transformer.fit_transform(np.array(list(summary.values())).reshape(-1, 1)).flatten()
cmap = linear.viridis
quantile_summary = 265 * (quantile_summary - min(quantile_summary)) / (max(quantile_summary) - min(quantile_summary))
quantile_summary = [stretch(i) for i in quantile_summary]
quantile_summary = dict(zip(summary.keys(), quantile_summary))